In [7]:
import pandas as pd  # importa a biblioteca pandas para manipulação de dados em formato tabular

df = pd.read_csv("homeOffice.csv")  # lê o arquivo CSV 'meditacao.csv' e cria um DataFrame chamado df

print(df.head())  # imprime as 5 primeiras linhas do DataFrame para visualizar o conteúdo inicial


   id_funcionario  idade         pais                      cargo  \
0               1     34       Brasil       Analista de Sistemas   
1               2     29          EUA           Designer Gráfico   
2               3     41        Índia        Gerente de Projetos   
3               4     37  Reino Unido  Especialista em Marketing   
4               5     25       Canadá            Suporte Técnico   

   horas_trabalho_diario  nivel_satisfacao  \
0                      8                 7   
1                      7                 8   
2                      9                 6   
3                      8                 9   
4                      7                 5   

                                     resposta_aberta  
0  Trabalho de casa tem sido um desafio enorme. N...  
1  Gosto da flexibilidade que o home office me of...  
2  A comunicação à distância exige mais planejame...  
3  O home office melhorou muito minha qualidade d...  
4  Sinto que a rotina em casa é menos e

In [8]:
import spacy  # importa a biblioteca spaCy para processamento de linguagem natural

nlp = spacy.load("pt_core_news_sm")  # carrega o modelo de linguagem para português (pequeno)

def preprocess(text):
    if not isinstance(text, str):  # verifica se o input é uma string, senão retorna string vazia
        return ""
    doc = nlp(text.lower())  # converte o texto para minúsculas e processa com spaCy
    # cria uma lista de lemas (forma base das palavras) filtrando tokens que são letras e não são stopwords
    tokens = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]  
    print(tokens)  # imprime os tokens lematizados para conferência
    return " ".join(tokens)  # retorna os tokens unidos em uma string novamente

# aplica a função preprocess em cada texto da coluna 'resposta_aberta' e cria uma nova coluna 'processed_text'
df["processed_text"] = df["resposta_aberta"].apply(preprocess)

# imprime as primeiras linhas das colunas originais e processadas para comparar
print(df[["resposta_aberta", "processed_text"]].head())


['trabalho', 'casa', 'ser', 'desafio', 'enorme', 'início', 'difícil', 'manter', 'concentração', 'haver', 'muito', 'distração', 'família', 'televisão', 'barulho', 'criei', 'rotina', 'manter', 'foco', 'sinto', 'interação', 'presencial', 'sinto', 'comunicação', 'mensagem', 'substituir', 'reunião', 'face', 'face', 'causa', 'tentar', 'técnica', 'produtividade', 'método', 'pomodoro', 'melhorar', 'desempenho']
['gostar', 'flexibilidade', 'home', 'office', 'oferecer', 'gerenciar', 'eficaz', 'enfrentar', 'dificuldade', 'separar', 'vida', 'pessoal', 'profissional', 'trabalhar', 'dever', 'ergonomia', 'estação', 'trabalho', 'ideal', 'causar', 'dor', 'costa', 'tentar', 'pausa', 'regular', 'alongamento', 'com si', 'disciplinar']
['comunicação', 'distância', 'exigir', 'planejamento', 'organização', 'reunião', 'virtual', 'constante', 'sinto', 'sobrecarregar', 'quantidade', 'informação', 'tecnologia', 'ajudar', 'fonte', 'estresse', 'ocorrer', 'problema', 'conexão', 'tentar', 'melhorar', 'gestão', 'equi

In [9]:
# como esses textos sao grandes, a melhor estrategia é trabalhar com chunks,
# dessa forma podemos trabalhar melhor com limite de tokens e melhorar o desempenho computacional

from langchain.text_splitter import CharacterTextSplitter  # importa o splitter de texto do LangChain

# Junta todos os textos processados num só texto longo
texto_longo = " ".join(df["processed_text"].tolist())

# Inicializa o splitter com tamanho do chunk de 100 caracteres e sobreposição de 20 caracteres
splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=20, separator=" ")

# Quebra o texto longo em pedaços (chunks) de tamanho definido
chunks = splitter.split_text(texto_longo)

# Imprime a lista de chunks para visualização
print(chunks)


['trabalho casa ser desafio enorme início difícil manter concentração haver muito distração família', 'distração família televisão barulho criei rotina manter foco sinto interação presencial sinto', 'presencial sinto comunicação mensagem substituir reunião face face causa tentar técnica', 'causa tentar técnica produtividade método pomodoro melhorar desempenho gostar flexibilidade home', 'flexibilidade home office oferecer gerenciar eficaz enfrentar dificuldade separar vida pessoal', 'separar vida pessoal profissional trabalhar dever ergonomia estação trabalho ideal causar dor costa', 'causar dor costa tentar pausa regular alongamento com si disciplinar comunicação distância exigir', 'distância exigir planejamento organização reunião virtual constante sinto sobrecarregar quantidade', 'quantidade informação tecnologia ajudar fonte estresse ocorrer problema conexão tentar melhorar', 'tentar melhorar gestão equipe ferramenta colaborativo processo gradual home office melhorar', 'home office

In [10]:
from sentence_transformers import SentenceTransformer  # importa a biblioteca para gerar embeddings

# Carrega o modelo pré-treinado Multilingual MiniLM para gerar embeddings em múltiplos idiomas, inclusive português
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Imprime a quantidade total de chunks que temos para processar
print(f"Total chunks: {len(chunks)}")

# Gera embeddings vetoriais para cada chunk de texto na lista "chunks"
embeddings_chunks = model.encode(chunks)

# Imprime quantos embeddings foram gerados e a dimensão (tamanho do vetor) do primeiro embedding
print(f"Gerados {len(embeddings_chunks)} embeddings para chunks, dimensão {embeddings_chunks[0].shape}")


Total chunks: 16
Gerados 16 embeddings para chunks, dimensão (384,)


In [11]:
import chromadb
from chromadb.config import Settings

# Inicializa o cliente do ChromaDB com configurações padrão
client = chromadb.Client(Settings())

# Cria ou abre uma coleção chamada "teste2" para armazenar dados (embeddings + documentos)
collection = client.get_or_create_collection(name="teste2")

# Cria uma lista de IDs para cada chunk, convertendo os índices para string (ex: '0', '1', '2', ...)
ids = [str(i) for i in range(len(chunks))]

# Cria uma lista de metadados para cada chunk, contendo a origem e o índice do chunk
metadatas = [{"source": "resposta_aberta", "chunk_index": i} for i in range(len(chunks))]

# Adiciona os dados na coleção:
# - ids: identificadores únicos para cada documento
# - documents: os chunks de texto
# - embeddings: os vetores gerados para cada chunk
# - metadatas: informações extras sobre cada chunk para ajudar na consulta e organização
collection.add(
    ids=ids,
    documents=chunks,
    embeddings=embeddings_chunks,
    metadatas=metadatas
)


In [12]:
# Texto da consulta que queremos buscar no banco de dados
query_text = "Como manter foco e concentração no trabalho remoto?"

# Gera o embedding (vetor) para o texto da consulta usando o mesmo modelo de embeddings
query_embedding = model.encode([query_text])[0]  # model.encode retorna lista, pegamos o primeiro vetor

# Faz a consulta no ChromaDB para buscar os 3 chunks mais similares ao embedding da query
results = collection.query(
    query_embeddings=[query_embedding],  # lista com o vetor da query
    n_results=3,                         # queremos os 3 resultados mais próximos
    include=['documents', 'distances', 'metadatas']  # queremos os textos, a distância e os metadados retornados
)

print("Top 3 resultados mais similares:")
# Itera sobre os documentos, distâncias e metadados dos resultados para mostrar
for doc, dist, meta in zip(results['documents'][0], results['distances'][0], results['metadatas'][0]):
    print(f"Distância: {dist:.4f} | Meta: {meta} | Texto: {doc}\n")


Top 3 resultados mais similares:
Distância: 13.3198 | Meta: {'source': 'resposta_aberta', 'chunk_index': 13} | Texto: estruturar afetar motivação trabalho horário irregular sinto dificuldade desconectar dia contato

Distância: 13.4542 | Meta: {'chunk_index': 0, 'source': 'resposta_aberta'} | Texto: trabalho casa ser desafio enorme início difícil manter concentração haver muito distração família televisão barulho criei rotina manter foco sinto interação presencial sinto comunicação mensagem substituir reunião face face causa tentar técnica produtividade método pomodoro melhorar desempenho gostar flexibilidade home office oferecer gerenciar eficaz enfrentar dificuldade separar vida pessoal profissional trabalhar dever ergonomia estação trabalho ideal causar dor costa tentar pausa regular alongamento com si disciplinar comunicação distância exigir planejamento organização reunião virtual constante sinto sobrecarregar quantidade informação tecnologia ajudar fonte estresse ocorrer problema 